# Estimación de ICI mediante histogramas de diagramas de constelación

## Tareas
- [X] Hacerlo 2D
- [ ] Con ambos utilizar un clasificador
- [ ] En caso tal, variar los bins

## Librerías

In [93]:
import sofa
import polars as pl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow.keras as ker
import json
import os

from scipy.io import loadmat
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import LogNorm

from sklearn.mixture import GaussianMixture
from sklearn.metrics import (accuracy_score, f1_score, multilabel_confusion_matrix,
                             ConfusionMatrixDisplay)
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import models, regularizers, Sequential, utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from itertools import product
from time import time

## Funciones globales

In [94]:
# Función especial para leer todos los datos con la estructura estudiada
def read_data(folder_rx):
    data = {}

    # Leer la carpeta principal
    for folder in os.listdir(folder_rx):
        # Leer las subcarpetas
        if folder.endswith("spacing"):
            data[folder] = {}
            for file in os.listdir(f"{folder_rx}/{folder}"):
                if file.find("consY") != -1:
                    data_name = file.split("_")[2]
                    if data[folder].get(data_name) == None:
                        data[folder][data_name] = {}
                    mat_file_data = loadmat(f"{folder_rx}/{folder}/{file}")
                    data[folder][data_name] = mat_file_data
    return data

def plot_constellation_diagram(X, ax):
    ax.scatter(X.real, X.imag, alpha=0.5)
    ax.set_title("Constellation diagram")
    ax.set_xlabel("I")
    ax.set_ylabel("Q")
    
def calculate_gmm(data, gm_kwargs):
    return GaussianMixture(**gm_kwargs).fit(data)
    
def calculate_1d_histogram(X, bins):
    hist_y, hist_x = np.histogram(X.real, bins=bins)
    # Remove last bin edge
    hist_x = hist_x[:-1]
    
    return hist_x, hist_y

def plot_1d_histogram(X, ax):
    ax.hist(X, bins=bins, density=True, alpha=0.5, label="Calculated histogram")
    
def plot_gmm_1d(gm, limits):
    x = np.linspace(*limits, 1000)
    
    logprob = gm.score_samples(x.reshape(-1, 1))
    responsibilities = gm.predict_proba(x.reshape(-1, 1))
    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]
    
    ax.plot(x, pdf_individual, '--', label="Adjusted histogram")

def plot_gmm_2d(gm, limits, ax):
    x = y = np.linspace(*limits)
    X, Y = np.meshgrid(x, y)
    Z = -gm.score_samples(np.array([X.ravel(), Y.ravel()]).T).reshape(X.shape)

    ax.contour(
        X, Y, Z,
        norm=LogNorm(vmin=1.0, vmax=1000.0), 
        levels=np.logspace(0, 3, 25), cmap="seismic"
    )
    
def calculate_3d_histogram(X, bins, limits, spacing, snr):
    hist, xedges, yedges = np.histogram2d(X.real, X.imag, bins=bins, range=[[*limits], [*limits]])

    # Define the extent
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

    # Create the meshgrid for the surface plot, excluding the last edge
    x_mesh, y_mesh = np.meshgrid(xedges[:-1], yedges[:-1])
    
    return hist, x_mesh, y_mesh
    
def plot_3d_histogram(x_mesh, y_mesh, hist, ax):
    ax.plot_surface(x_mesh, y_mesh, hist.T, cmap="seismic", rstride=1, cstride=1, edgecolor="none")
    ax.set_title("3D Histogram")
    ax.set_xlabel("I")
    ax.set_ylabel("Q")

In [95]:
def calc_once(varname, fn, args):
    """ Calculate a variable only once. """
    if varname not in globals():
        return fn(**args)
    return eval(varname)

def classificator(df, interval_lst, column_name):
    """Transforms a dataframe's column into classes"""
    array = df[column_name].to_numpy()
    indexes_lst = []
    for i, interval in enumerate(interval_lst):
        lower_limit, upper_limit = interval
        indexes_lst.append(np.intersect1d(np.where(lower_limit < array), np.where(array <= upper_limit)))
    
    classfull = df[column_name]
    for index, indexes in enumerate(indexes_lst):
        classfull[indexes] = index

    df_classfull = df.clone()
    df_classfull.replace(column_name, classfull)
    
    return df_classfull

def classifier_model(input_dim, layers_props_lst, classes_n, loss_fn):
    """ Compile a sequential model for classification purposes. """
    model = ker.Sequential()
    # Hidden layers
    for i, layer_props in enumerate(layers_props_lst):
        if i == 0:
            model.add(ker.layers.Dense(**layer_props, input_dim=input_dim))
        else:
            model.add(ker.layers.Dense(**layer_props))
    # Classifier
    model.add(ker.layers.Dense(units=classes_n, activation="softmax"))

    model.compile(loss=loss_fn, optimizer="adam")
    return model    


def classification_crossvalidation(X, y, n_splits, layer_props, classes_n, loss_fn, callbacks):
    """ Crossvalidation of a classification network. """
    # Store initial time
    t0 = time()

    # Scores dict
    scores = {}
    scores["loss"] = []
    scores["acc"] = {"train": [], "test": []}
    scores["f1"] = {"train": [], "test": []}
    scores["cm"] = {"train": [], "test": []}
    
    # K-fold crossvalidation
    kf = KFold(n_splits=n_splits, shuffle=True)

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Input variables standarizer
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test_kf = sc.transform(X_test)
        
        model = classifier_model(X_train.shape[1], layer_props, classes_n, loss_fn)
        
        # Save test scalar loss
        if callbacks:
            loss = model.fit(
                X_train, y_train, epochs=5000, batch_size=64, verbose=0, callbacks=callbacks
            )
        else:
            loss = model.fit(X_train, y_train, epochs=5000, batch_size=64, verbose=0)
        print(f"Needed iterations: {len(loss.history['loss'])}")
        
        # Predict using train values
        fuzzy_predictions_train = model.predict(X_train)
        # Predict using test values
        fuzzy_predictions_test = model.predict(X_test_kf)
        
        # Assign class based on higher probability in membership vector
        predictions_train = np.array([np.argmax(fuzzy_prediction) for fuzzy_prediction in fuzzy_predictions_train])
        predictions_test = np.array([np.argmax(fuzzy_prediction) for fuzzy_prediction in fuzzy_predictions_test])

        # Dataframe for better visualization
        train_data_train = pl.DataFrame(
            {"ICI": [y_train], "Predicted ICI": [predictions_train]}
        )
        train_data_test = pl.DataFrame(
            {"ICI": [y_test], "Predicted ICI": [predictions_test]}
        )

        # Accuracy
        acc_score_train = accuracy_score(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        acc_score_test = accuracy_score(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )

        # F1
        f1_score_train = f1_score(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"],
            average="micro"
        )
        f1_score_test = f1_score(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"],
            average="micro"
        )
         
        # Confusion Matrix
        cm_score_train = multilabel_confusion_matrix(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        cm_score_test = multilabel_confusion_matrix(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )

        # Append to lists
        scores["loss"].append(loss)
        scores["acc"]["train"].append(acc_score_train)
        scores["acc"]["test"].append(acc_score_test)
        scores["f1"]["train"].append(f1_score_train)
        scores["f1"]["test"].append(f1_score_test)
        scores["cm"]["train"].append(cm_score_train)
        scores["cm"]["test"].append(cm_score_test)
        
    print(f"Time elapsed: {(time() - t0)/60:.2f} minutes")

    return scores


def test_classification_model(data, n_splits, max_neurons, activations, classes_n,
                               use_osnr=True, loss_fn="sparse_categorical_crossentropy"):
    """ Test a spectral overlapping classification model with given parameters. """
    # Set variable number
    var_n = 17 if use_osnr else 16
    
    # Split variables
    # Variables
    X = np.array(data[:, 0:var_n]).T
    # Tags
    y = np.array(data[:, 19:20]).T
    
    # Layer properties
    layer_props = [
        {"units": max_neurons // (2**i), "activation": activation}
        for i, activation in enumerate(activations)
    ]
    print(layer_props)
    callbacks = [
        EarlyStopping(monitor="loss", patience=30, mode="min", restore_best_weights=True)
    ]
    
    return classification_crossvalidation(X, y, n_splits, layer_props, classes_n, loss_fn, callbacks)

def plot_classes_scores(scores, scenario):
    score_names = ["acc"]
    data_type = ["train", "test"]
    markers = ["o", "D", "o", "D"]
    colors = ["dodgerblue", "dodgerblue", "red", "red"]
    
    # Plot loss
    plot_losses(scores, scenario, 
                lambda index: f"{'FCM' if index%2==0 else 'GKM'} {'B2B' if index<2 else 'optical fiber at ' + ('0' if index<4 else '9') + ' dBm'}",
                based_on_index=True)
    
    # Plot scores
    plot_scores(scores, ["B2B", "0 dBm", "9 dBm"], scenario, score_names,
                data_type, label=lambda index: f"{'FCM' if index%2==0 else 'GKM'}",
                xlabel="Scenario", markers=markers, colors=colors, multiple_points=True,
                based_on_index=True)
    plt.show()
    
def plot_cm(scores, interval_lst):
    CM = np.array(scores.get("cm").get("test"))
    for n, interval in enumerate(interval_lst):
        result = np.zeros(CM[0][0].shape)
        for cm in CM:
            result = np.add(result, cm[n])
        result /= np.sum(result)
        disp = ConfusionMatrixDisplay(confusion_matrix=result, display_labels=["Positive", "Negative"])
        disp.plot(colorbar=False)
        lower_limit, upper_limit = interval 
        plt.title(f"Confusion matrix for class from {lower_limit} GHz up to {upper_limit} GHz")
        plt.show()

## Leer datos

In [96]:
file_tx = "Demodulation/Data/2x16QAM_16GBd.mat"
folder_rx = "Demodulation/Data/"

# Transmitted data
X_tx_norm = loadmat(file_tx)
X_tx_norm = X_tx_norm.get("Constellation").flatten()[0][0].flatten()
X_tx = sofa.mod_norm(X_tx_norm, 10)*X_tx_norm

# Read received data
data = read_data(folder_rx)

## Obtener histogramas

In [97]:
plot = False
spacings = ["15", "15.5", "16", "16.5", "17", "17.6", "18"]

histograms = {}
gaussians = {}
bins = 128
limits = [-5, 5]

for spacing in spacings:
    X_rx = data[f"{spacing}GHz_spacing"]
    for snr in X_rx:
        # Extract data 
        X_ch_norm = X_rx[snr].get("const_Y").flatten()
        X_ch = sofa.mod_norm(X_ch_norm, 10)*X_ch_norm

        if plot:
            plt.figure(figsize=(12, 12), layout="tight")

            # Plot constellation diagram
            ax = plt.subplot(2, 2, 1)
            plot_constellation_diagram(X_ch, ax)
        
        # Calculate 2D GMM
        input_data = np.vstack((X_ch.real, X_ch.imag)).T
        gm_kwargs = {
            "means_init": np.array(list(product([-3, -1, 1, 3], repeat=2))), 
            "n_components": 16
        }
        gm_2d = calculate_gmm(input_data, gm_kwargs)
        
        if plot:
            # Plot 2D GMM
            plot_gmm_2d(gm_2d, limits, ax)
            ax.grid(True)

        # Calculate 3D histogram
        hist, x_mesh, y_mesh = calculate_3d_histogram(X_ch, bins, limits, spacing, snr)
        
        # Save 3D histogram
        if histograms.get(f"{spacing}GHz_spacing") == None:
            histograms[f"{spacing}GHz_spacing"] = {}
        histograms[f"{spacing}GHz_spacing"][snr[5:]] = hist
    
        if plot:
            # Plot 3D histogram
            ax = plt.subplot(2, 2, 2, projection="3d")
            plot_3d_histogram(x_mesh, y_mesh, hist, ax)

        # Plot I and Q histograms separately
        # I
        if plot:
            ax = plt.subplot(2, 2, 3)
            plot_1d_histogram(X_ch.real, ax)
        
        hist_x, hist_y = calculate_1d_histogram(X_ch.real, bins)
        input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
        gm_kwargs = {
            "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
            "n_components": 4
        }
        gm_i = calculate_gmm(input_data, gm_kwargs)
        if plot:
            plot_gmm_1d(gm_i, limits)
        
            ax.set_title("I-Histogram")
            ax.set_xlabel("I")
            ax.grid(True)

        # Q
        if plot:
            ax = plt.subplot(2, 2, 4)
            plot_1d_histogram(X_ch.imag, ax)
        
        hist_x, hist_y = calculate_1d_histogram(X_ch.imag, bins)
        input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
        gm_kwargs = {
            "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
            "n_components": 4
        }
        gm_q = calculate_gmm(input_data, gm_kwargs)
        if plot:
            plot_gmm_1d(gm_q, limits)
            
            ax.set_title("Q-Histogram")
            ax.set_xlabel("Q")
            ax.grid(True)
    
        # Save gaussians
        if gaussians.get(f"{spacing}GHz_spacing") == None:
            gaussians[f"{spacing}GHz_spacing"] = {}
        gaussians[f"{spacing}GHz_spacing"][snr[5:]] = [gm_2d, gm_i, gm_q]

        if plot:
            plt.suptitle(f"Plots for {snr[5:]} OSNR and {spacing} GHz of spacing")

            plt.show()

In [ ]:
spacings = ["15", "15.5", "16", "16.5", "17", "17.6", "18"]

for spacing in spacings:
    X_rx = data[f"{spacing}GHz_spacing"]
    for snr in X_rx:
        # Extract data 
        X_ch_norm = X_rx[snr].get("const_Y").flatten()
        X_ch = sofa.mod_norm(X_ch_norm, 10)*X_ch_norm

        plt.figure(figsize=(12, 12), layout="tight")

        # Plot constellation diagram
        ax = plt.subplot(2, 2, 1)
        plot_constellation_diagram(X_ch, ax)
        
        gm_2d = gaussians.get(f"{spacing}GHz_spacing").get(f"{snr[5:]}")[0]
        
        # Plot 2D GMM
        plot_gmm_2d(gm_2d, limits, ax)
        ax.grid(True)

        # Calculate 3D histogram
        hist, x_mesh, y_mesh = calculate_3d_histogram(X_ch, bins, limits, spacing, snr)
        
        # Save 3D histogram
        if histograms.get(f"{spacing}GHz_spacing") == None:
            histograms[f"{spacing}GHz_spacing"] = {}
        histograms[f"{spacing}GHz_spacing"][snr[5:]] = hist
    
        if plot:
            # Plot 3D histogram
            ax = plt.subplot(2, 2, 2, projection="3d")
            plot_3d_histogram(x_mesh, y_mesh, hist, ax)

        # Plot I and Q histograms separately
        # I
        if plot:
            ax = plt.subplot(2, 2, 3)
            plot_1d_histogram(X_ch.real, ax)
        
        hist_x, hist_y = calculate_1d_histogram(X_ch.real, bins)
        input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
        gm_kwargs = {
            "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
            "n_components": 4
        }
        gm_i = calculate_gmm(input_data, gm_kwargs)
        if plot:
            plot_gmm_1d(gm_i, limits)
        
            ax.set_title("I-Histogram")
            ax.set_xlabel("I")
            ax.grid(True)

        # Q
        if plot:
            ax = plt.subplot(2, 2, 4)
            plot_1d_histogram(X_ch.imag, ax)
        
        hist_x, hist_y = calculate_1d_histogram(X_ch.imag, bins)
        input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
        gm_kwargs = {
            "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
            "n_components": 4
        }
        gm_q = calculate_gmm(input_data, gm_kwargs)
        if plot:
            plot_gmm_1d(gm_q, limits)
            
            ax.set_title("Q-Histogram")
            ax.set_xlabel("Q")
            ax.grid(True)
    
        # Save gaussians
        if gaussians.get(f"{spacing}GHz_spacing") == None:
            gaussians[f"{spacing}GHz_spacing"] = {}
        gaussians[f"{spacing}GHz_spacing"][snr[5:]] = [gm_2d, gm_i, gm_q]

        if plot:
            plt.suptitle(f"Plots for {snr[5:]} OSNR and {spacing} GHz of spacing")

            plt.show()

## Preparar datos

In [119]:
# Dataframe con 98 columnas
# 16 primeras para las medias
# 64 siguientes para los valores de las matrices de covarianza
# Penúltima para el valor del OSNR (dB)
# Última para el valor del espaciamiento (GHz)

df_dict = {f"col{n}": [] for n in range(98)}

# Iterate over the dictionary and populate the DataFrame
for spacing, osnr in gaussians.items():
    for osnr_, gmm_data in osnr.items():
        gmm = gmm_data[0]
        means = gmm.means_.flatten()
        covariances = gmm.covariances_.flatten()
        osnr_value = np.array([float(osnr_[:-2])])
        spacing_value = np.array([float(spacing[:-11])])
        
        features = np.concatenate((means, covariances, osnr_value, spacing_value))
        
        for n in range(98):
            df_dict[f"col{n}"].append(features[n])
# Reset the index of the DataFrame
df = pl.DataFrame(df_dict)

# Print the DataFrame
print(df)
df.write_json("histograms.json")

shape: (70, 98)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬──────────┬───────┬───────┐
│ col0      ┆ col1      ┆ col2      ┆ col3      ┆ … ┆ col94     ┆ col95    ┆ col96 ┆ col97 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---      ┆ ---   ┆ ---   │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64      ┆ f64   ┆ f64   │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪══════════╪═══════╪═══════╡
│ -3.061791 ┆ -3.083884 ┆ -3.011383 ┆ -1.095477 ┆ … ┆ 0.003165  ┆ 0.263649 ┆ 35.0  ┆ 15.0  │
│ -2.79813  ┆ -2.852291 ┆ -2.848151 ┆ -0.888817 ┆ … ┆ -0.007277 ┆ 0.244642 ┆ 30.0  ┆ 15.0  │
│ -2.88227  ┆ -2.787026 ┆ -2.989467 ┆ -0.990089 ┆ … ┆ -0.000877 ┆ 0.300377 ┆ 25.0  ┆ 15.0  │
│ -2.940511 ┆ -2.963084 ┆ -2.917673 ┆ -1.102453 ┆ … ┆ 0.015277  ┆ 0.305211 ┆ 40.0  ┆ 15.0  │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …        ┆ …     ┆ …     │
│ -3.133732 ┆ -3.183743 ┆ -3.048301 ┆ -1.128827 ┆ … ┆ 

In [121]:
interval_lst = [(0, 17.6), (17.6, 18)]
df_2classes = classificator(df, interval_lst, "col97")
print(df_2classes)

shape: (70, 98)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬──────────┬───────┬───────┐
│ col0      ┆ col1      ┆ col2      ┆ col3      ┆ … ┆ col94     ┆ col95    ┆ col96 ┆ col97 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---      ┆ ---   ┆ ---   │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64      ┆ f64   ┆ f64   │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪══════════╪═══════╪═══════╡
│ -3.061791 ┆ -3.083884 ┆ -3.011383 ┆ -1.095477 ┆ … ┆ 0.003165  ┆ 0.263649 ┆ 35.0  ┆ 0.0   │
│ -2.79813  ┆ -2.852291 ┆ -2.848151 ┆ -0.888817 ┆ … ┆ -0.007277 ┆ 0.244642 ┆ 30.0  ┆ 0.0   │
│ -2.88227  ┆ -2.787026 ┆ -2.989467 ┆ -0.990089 ┆ … ┆ -0.000877 ┆ 0.300377 ┆ 25.0  ┆ 0.0   │
│ -2.940511 ┆ -2.963084 ┆ -2.917673 ┆ -1.102453 ┆ … ┆ 0.015277  ┆ 0.305211 ┆ 40.0  ┆ 0.0   │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …        ┆ …     ┆ …     │
│ -3.133732 ┆ -3.183743 ┆ -3.048301 ┆ -1.128827 ┆ … ┆ 